In [ ]:
import numpy as np
from wheel_library import wheel_from_name
import bikewheellib as bl
from IPython.display import display

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
rad_only = [True, False, False]

wheels = [wheel_from_name('std_elbows_in') for i in range(3)]

wheels[0].lace_radial(n_spokes=len(wheels[0].spokes),
                      diameter=wheels[0].spokes[0].diameter,
                      young_mod=wheels[0].spokes[0].young_mod,
                      offset=0.0)
wheels[0].apply_tension(800)

wheels[1].lace_cross(n_cross=1,
                     n_spokes=len(wheels[1].spokes),
                     diameter=wheels[1].spokes[0].diameter,
                     young_mod=wheels[1].spokes[0].young_mod,
                     offset=0.0)
wheels[1].apply_tension(800)

wheels[2].lace_cross(n_cross=3,
                     n_spokes=len(wheels[2].spokes),
                     diameter=wheels[2].spokes[0].diameter,
                     young_mod=wheels[2].spokes[0].young_mod,
                     offset=0.0)
wheels[2].apply_tension(800)

In [ ]:
fig = plt.figure(figsize=(6.5, 5.0))

draw_opts = {'spk_ls_width': 0.5, 'spk_ds_width': 0.5, 'rim_width': 1.0}

# Spoke numbers
s_num = np.arange(36)

with plt.style.context('seaborn-paper'):

    fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(6.5, 5.0), gridspec_kw={'width_ratios': [1, 2.5, 2.5, 2.5]})
    
    for i in range(len(wheels)):

        rr = bl.ModeMatrix(wheels[i], N=36)
        K = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=True)

        # Draw wheel
        wheels[i].draw(ax[i, 0], draw_opts)

        # Calculate change in spoke tension for radial load
        F_ext = rr.calc_F_ext(np.array([np.pi]),
                          np.array([[0., 1., 0., 0.]]))
        d = np.linalg.solve(K, F_ext)

        delta_T = [-s.EA/s.length * s.n.dot(rr.B_theta(s.rim_pt[1], [0, 1, 2]).dot(d))
                   for s in wheels[i].spokes]

        ax[i, 1].bar(s_num, delta_T, width=0.6, color='#1F77B4', edgecolor='none', alpha=0.6)
        ax[i, 1].bar(18, delta_T[18], width=0.6, color='#D62728', edgecolor='none')


        if not rad_only[i]:
            # Calculate change in spoke tension for unit tangential load
            F_ext = rr.calc_F_ext(np.array([np.pi]),
                              np.array([[0., 0., 1., 0.]]))
            d = np.linalg.solve(K, F_ext)

            delta_T = [-s.EA/s.length * s.n.dot(rr.B_theta(s.rim_pt[1], [0, 1, 2]).dot(d))
                       for s in wheels[i].spokes]

            ax[i, 2].bar(s_num, delta_T, width=0.6, color='#1F77B4', edgecolor='none', alpha=0.6)
            ax[i, 2].bar(18, delta_T[18], width=0.6, color='#D62728', edgecolor='none')

            # Calculate change in spoke tension for a combined loading case
            F_ext = rr.calc_F_ext(np.array([np.pi]),
                              np.array([[0., 500., 50., 0.]]))
            d = np.linalg.solve(K, F_ext)

            delta_T = [-s.EA/s.length * s.n.dot(rr.B_theta(s.rim_pt[1], [0, 1, 2]).dot(d))
                       for s in wheels[i].spokes]

            ax[i, 3].bar(s_num, delta_T, width=0.6, color='#1F77B4', edgecolor='none', alpha=0.6)
            ax[i, 3].bar(18, delta_T[18], width=0.6, color='#D62728', edgecolor='none')

        for j in range(1, 4):
            ax[i, j].set_xlim([-1, 36])
            ax[i, j].set_xticks([])

        ax[i, 1].set_ylim(-0.5, 0.1)
        ax[i, 1].set_yticks([-.5, -.4, -.3, -.2, -.1, 0., 0.1])
        ax[i, 1].set_yticklabels(['-0.5', '', '', '', '', '0', ''])

        ax[i, 2].set_ylim(-0.5, 0.5)
        ax[i, 2].set_yticks(np.arange(-0.5, 0.51, 0.1))
        ax[i, 2].set_yticklabels(['-0.5', '', '', '', '', '0', '', '', '', '', '0.5'])

        ax[i, 3].set_ylim(-200, 50)
        ax[i, 3].set_yticks([-200, -150, -100, -50, 0, 50])

    plt.tight_layout()
    plt.savefig('../figs/stress_analysis/_python_rad_tan_Tinf.pdf')